# Analysis of observed evapotranspiration TransRegio 2009  (step 2)

## Intro
The exercise of today is the second step in a series of three. The logic of this step is very similar to that of the first step. 

The final objective is to determine the actual evapotranspiration of the Hupsel catchment in the first weeks of May 2022. Since we do not have direct observations of the current surface fluxes (either eddy-covariance, lysimeters or through scintillometry) we split the process in two major steps (left and right in the figure below):
a. understand how the ET in Hupsel catchment responds to external forcings, based on historical data from Hupsel and elswhere
b. use that process understanding to make the best possible estimate of the ET of the Hupsel catchment in the past weeks.

The current land-use in the Hupsel catchment can be simplified as a mixture of grass and bare soil (the maize is just emerging). In the previous session we focussed on understanding the response of grass ET to external forcings (step 1). Now  we will study bare soil evaporation (step 2). On the Monday in week 4 we will finish step 3 and 4.

The concept of reference evapotranspiration is dealt with extensively in the [book used for Atmosphere Vegetation Soil Interactions](https://www-cambridge-org.ezproxy.library.wur.nl/core/books/transport-in-the-atmospherevegetationsoil-continuum/5944F8B7ADAC6409AD4575642431B2DC) (chapter 7 and 8). A summary of the [most essential concepts](reference_ET_concept.pdf) is also available.

Collect your answers in the <a href="Actual_ET_2-answer-sheet.docx" download>answer sheet</a>.

<img src="analysis-overview.png" width="80%">

## The method
The logic of today's practical is that reference ET is supposed to contain all main meteorological drivers of evapotranspiration. By comparing the reference ET with the observed actual ET (based on eddy-covariance measurements) we can find out if there are additional external factors that need to be taken into account. 

If all relevant information would be contained in reference ET, the crop factor (ET<sub>act</sub> / ET<sub>ref</sub>) would be constant. If it varies in time, that is an indication that additional factors play a role.

In the four steps sketched above we will focus on daily mean data (i.e. data that have been averaged over 24 hours).

## The data
To understand the response of bare soil evaporation to external forcings we will make use of flux observations obtained in August 2009 near [Merken, Germany](https://goo.gl/maps/qCNqUPkaDWbZVQhT7)  (see [Van Kesteren et al. (2013)](https://doi.org/10.1016/j.agrformet.2013.10.010). We choose this data set since it combines simultaneous flux observations over bare soil and sugar beet, over a period that contains a number of dry days, a rain event and the dry-down after that rain event.  

In the figure below you see the energy balance terms for the bare soil plot (left) and the sugar beet plot (right) for the same seven days. In the night between DOY 219 and 220 a significant amount of precipitation occurred.

<img src="EB_baresoil.png" width="100%">

## Initialize Python stuff and read the data
Please run the cell below by selecting it and pressing Shift+Enter. Or Press the Run button in the toolbar at the top of the screen (with the right pointing triangle).

In [1]:
# Load some necessary Python modules
import pandas as pd # Pandas is a library for data analysis
import numpy as np # Numpy is a library for processing multi-dimensional datasets
from hupsel_helper import myplot, myreadfile
from hupsel_helper import f_Lv, f_esat, f_s, f_makkink

Now read the data from the Excel file.

In [2]:
# File name
fname='TR32_WetDry_dataset.xlsx'

# Get the data
df = myreadfile(fname)

## Explore the data
### Information available in the dataframe
Before you start making computations with the data it is wise to first explore the data. 

To show the names of the available variables, type `df.keys()` in the cell below (and run, or press Shift+Enter). Based on the names you can also make the distinction between the two parts of the dataset:
* Bare soil data: variables with a subscript `_b`.
* Sugarbeet data: variables with a subscript `_s`.
Note that some of the variables you may need in your computations may be absent. Try to make a reasonable assumption for them (such is life when you get a dataset from someone: they're never perfect!).

The dataframe also contains information about the units of the variables: type `df.attrs['units']` in the cell below. You can also access the units of an individual variable as follows: `df.attrs['units']['u_10']` should give `[m/s]`. Finally, the dataframe also contains a more complete description of the variables: `df.attrs['description']`.

### Inspect the data
There are a number of ways to inspect the data (same info as in notebook for step 1):
* print the full dataframe in a cell (simply type `df` and run the cell)
* print a single (or a few) variable(s) from the dataframe (type for instance `df[['K_in_b']]` or `df[['K_in_b','T_b']]` to show the values of global radiation and temperature. Note the double square brackets which ensure a nicely printer table.
* plot the data with the plot command `myplot`

The plot command works as follows:
* plot one series as a line: `myplot(df,['Date','K_in_b'])`. So you need to provide the dataframe that contains all the data `df`) and you need to give the names of the variables to be used as `x` and `y` in the plot. The x-name and y-name are given as a list (enclosed in square brackets).
* plot two series as lines: `myplot(df,['Date','K_in_b'], ['Date','K_out_s'])`. The second series is also given as a list, separated from the first list with comma.
* plot a series with something other than a line 
  * `myplot(df,['Date','K_in_b','-'])`: plot a line
  * `myplot(df,['Date','K_in_b','o'])`: plot dots
  * `myplot(df,['Date','K_in_b','#'])`: plot a bar graph (can only be done for one series per graph)
* you can also plot series without using a dataframe (assume x, y and z are  arrays): `myplot([x,y],[x,z])`
       

### <span style='background:lightblue'>Question 1</span>
Characterize the weather conditions during the period in which the data were gathered. Since the dataset is rather short, and things change fast around the rainfall event, you should do this day-by-day. Use this to get an idea of the conditions during the 7 days of observations.

### <span style='background:lightblue'>Question 2</span>
The next steps is to look at the fluxes. Explore how: 
* the fluxes differ between the bare soil plot and the vegetated plot;
* the rainfall event affects the fluxes and that that response differs between the surfaces. 

Apart from the graphs you make yourself, you could also have a look at the graphs shown at the start of this notebook in the section 'The Data'.

Rather than looking at fluxes alone, it can also be helpful to focus on the partitioning between sensible and latent heat flux. For this you could look at variables like available energy (Q\*-G), Bowen ratio (H/L<sub>v</sub>E) and evaporative fraction (L<sub>v</sub>E/(Q\*-G)). For the period after the rain pay attention to how the partitioning changes between just after the rain and a few days after the rain. 

## Reference evapotranspiration
For reasons of simplicity and robstness, we will use the Makkink equation to determine the reference ET. The essential equations can be found in the [Formularium of Atmosphere-Vegetation-Soil Interactions](Forumularium_AVSI_2021.pdf). 

###  Reuse your functions
In practical for step 1 you developed a number of functions. Those are now directly available to you (note: they have been renamed to and now start with `f_`) :
In order to determine the reference ET in mm/day a number of ingredients are needed:
* `f_Lv(T)`: compte latent heat of vapourization from temperature (in K)
* `f_esat(T)`: compute saturated vapour pressure from temperature (in K)
* `f_s(T)`: compute the slope of the saturated vapour pressure as a function of temperature (in K)
* `f_gamma(T, p, q)`: compute the psychrometer constant from temperature (K), pressure (Pa) and specific humidity (kg/kg)
* `f_makkink(Kin, T, p, q)`: compute reference evapotranspiration according to the Makkink equation

### Determine the reference evapotranspiration
As in the first step, we will use the Makkink reference ET as a starting point for quantifying the meteorological drivers for evapotranspiration.

### <span style='background:lightblue'>Question 3</span>
Compute the reference evapotranspiration in mm/day based on the current data set. Check what is the unit of the flux you computed with your `f_makkink` function. Furthermore, check that the values that you compute make sense.

## Determine the actual evapotranspiraton
The actual evapotranspiration has been measured using the eddy-covariance technique. It is available in the data set in the variable named `LvE_b`for the bare soil plot.

### <span style='background:lightblue'>Question 4</span>
Compute the actual evapotranspiration in mm/day based on the eddy-covariance fluxes.

## Compare actual and reference evapotranspiration
Now that you have both actual and reference evapotranspiration availble, it is time to compare them. The first step would be to plot both in one graph

### <span style='background:lightblue'>Question 5</span>
How do actual and reference evapotranspiration compare? Are they identical, is there a fixed offset, or is the difference variable over time. If so, can you related those differences to specific conditions?

Part of the variability in the actual evapotranspiration is related to variations in meteorological conditions. Those variations are supposed be captured by the reference evapotranspiration. A straightforward way to see to what extent the reference evapotranspiration captures the variability of the actual evapotranspiration is to compute the crop factor (ET<sub>act</sub> / ET<sub>ref</sub>). 

### <span style='background:lightblue'>Question 6</span>
Compute the crop factor for the current data. What is the overall magnitude the crop factor? Is the crop factor constant over time, and if not, can you explain the variations (or at least bring forward a hypothesis)? It may also be interesting to look at the crop factor of sugarbeet (even though that surface is not our main interest now).

### <span style='background:lightblue'>Question 7</span>
Summarize your findings regarding the crop factor for bare soil in such a way that you could use it as simple model to derive bare soil evaporation from the reference ET. 
based on a limited number of variables (e.g. rainfall history, air humidity, temperature). There is no need to come with a model in the form of an equation, a look-up table is sufficient.

## Conclusion
You have made your second step to come to an estimation of the actual evapotranspiration of the Hupsel catchment in May 2022: you now know how the bare in the catchment might react to the external meteorological forcings (expressed in the reference evapotranspiration).

## Up to the next exercise
You have explored and quantified how the actual evapo(transpi)ration of grass and bare soil compare to the reference evapotranspiration. This information will help you to estimate the actual evapotranspiration of the Hupsel catchment for the current conditions (without having eddy-covariance measurements for this year).